### Anharmonic Oscillator with RBMs

We want to try creating an emulator using RBMs for the anharmonic oscillator problem from the CAT plot focus Challenge. 

For this problem, we are finding a reduced-basis for the Hamiltonian 
\begin{equation}
    H = -\frac{\partial^2}{\partial x^2} + \kappa_1 x^2 + \kappa_2 x^2.
\end{equation}

To begin, we start with importing the standard requisite packages.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.stats import qmc

def exact_solution(domain = (3,3), nsteps = 2000,params):
    

    
    d2_of_x = diags([1, -2, 1], [-1, 0, 1], shape=(n, n))  / ( dx ** 2 )

    H = -d2_of_x + diags( kappa_1 * mesh ** 2  +  kappa_2 * mesh ** 4).toarray()
    return eigenvalue, eigenvector